In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Please excuse the formatting, haven't really done much markdown before. Anyway, we get started by importing some libraries, and loading in our data.


In [0]:
import json
import re
import numpy as np    
import sklearn as sk
import scipy as sp
from sklearn import decomposition
from sklearn import ensemble
from nltk import PorterStemmer
from scipy import sparse
from sklearn.naive_bayes import BernoulliNB
from tensorflow import keras as ks
from tqdm import tqdm_notebook
import pickle

In [0]:

with open("drive/My Drive/gerald_data/train.json") as file:
  train_data = json.load(file)[::3]
#with open("drive/My Drive/gerald_data/test.json") as file:
#  test_data = json.load(file)[::10]
with open("drive/My Drive/gerald_data/stopwords.txt") as file:
  stopwords = file.read().splitlines()

# Preprocessing

Like in assignment 2, lowercase this bad boy and remove the punctuation. Then remove stopwords and apply stemming.

In [0]:
ps = PorterStemmer()
def preprocess(post):
  words = re.sub('[^a-z0-9 ]', '', post["post"].lower()).split()
  return list(map(ps.stem, filter(lambda w : w not in stopwords, words)))

posts = [preprocess(post) for post in train_data]

Now we create our list of unique words. However, we also want to discard the nonsense, so we delete words from the list that occur in too few posts. 

In [7]:
#TOO_RARE_THRESHOLD = 0.005  # Rate of occurence below which a word is too rare to be considered
NUM_WORDS=5000

words = dict()  # Match words to the number of posts they occured in.
for post in posts:
    for word in list(set(post)): # set trick removes duplicates
        if word in words:
             words[word] += 1
        else:
             words[word] = 1

print "Features before removal of rare words: %d" % len(words)                
#for word in words.keys():
#    if words[word] < TOO_RARE_THRESHOLD * len(posts):
#        del(words[word])
words = sorted(words.keys(), key=words.get, reverse=True)[:NUM_WORDS]    # words is now a list, sorted by how many times each word occured.    


print "After removal: %d" % len(words)



Features before removal of rare words: 434836
After removal: 2186


In [0]:
with open("drive/My Drive/gerald_data/vocab", "w") as file:
  pickle.dump(words, file)
with open("drive/My Drive/gerald_data/posts", "w") as file:
  pickle.dump(posts, file)  

In [0]:
MAXLEN=500

def vectorize_x(data):
  pruned_data = [filter(lambda w:w in words, post) for post in data]
  return ks.preprocessing.sequence.pad_sequences([[words.index(word) for word in post] for post in pruned_data], maxlen=MAXLEN)

def vectorize_gender(data):
  return np.array(map(lambda post : 1 if post["gender"] == "male" else 0, data))  

def vectorize_age(data):
  return np.array([post['age'] for post in data])  

X = vectorize_x(posts)  

In [0]:
model = ks.models.Sequential()

model.add(ks.layers.Embedding(len(words), 20, input_length=MAXLEN))
#model.add(ks.layers.Dropout(0.2))
#model.add(ks.layers.Flatten())
model.add(ks.layers.Conv1D(100, 3, padding='valid', activation='elu'))
model.add(ks.layers.Conv1D(100, 2, padding='valid', activation='elu'))
#model.add(ks.layers.Dropout(0.2))
model.add(ks.layers.Flatten())
model.add(ks.layers.Dense(200, activation='elu'))
model.add(ks.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
Y_gender = vectorize_gender(train_data)
#print(X.shape, Y_gender.shape, len(words))
model.summary()
#model.build()
model.fit(X, Y_gender, epochs=1, validation_split=0.1, batch_size=50)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 20)           43720     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 498, 100)          6100      
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 497, 100)          20100     
_________________________________________________________________
flatten_3 (Flatten)          (None, 49700)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 200)               9940200   
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 201       
Total params: 10,010,321
Trainable params: 10,010,321
Non-trainable params: 0
__________________________________________

In [18]:
with open("drive/My Drive/gerald_data/test.json") as file:
  test_data = json.load(file)

X_test = vectorize_x([preprocess(post) for post in test_data])
Y_test_gender = vectorize_gender(test_data)

model.evaluate(X_test, Y_test_gender)  
model.save("drive/My Drive/gerald_data/W_gender")

131703/131703 [==============================] - 20s 153us/sample - loss: 0.6332 - acc: 0.6379


In [0]:
model = ks.models.Sequential()

model.add(ks.layers.Embedding(len(words), 20, input_length=MAXLEN))
model.add(ks.layers.Dropout(0.2))
#model.add(ks.layers.Flatten())
model.add(ks.layers.Conv1D(150, 3, padding='valid', activation='elu'))
model.add(ks.layers.Conv1D(100, 3, padding='valid', activation='elu'))
model.add(ks.layers.Dropout(0.2))
model.add(ks.layers.Flatten())
model.add(ks.layers.Dense(200, activation='elu'))
model.add(ks.layers.Dense(1, activation='relu'))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])

In [21]:
Y_age = vectorize_age(train_data)
print(X.shape, Y_age.shape, len(words))
model.summary()
#model.build()
model.fit(X, Y_age, epochs=2, validation_split=0.1, batch_size=50)

((175604, 500), (175604,), 2186)
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 500, 20)           43720     
_________________________________________________________________
dropout_6 (Dropout)          (None, 500, 20)           0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 498, 150)          9150      
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 496, 100)          45100     
_________________________________________________________________
dropout_7 (Dropout)          (None, 496, 100)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 49600)             0         
_________________________________________________________________
dense_10 (Dense)     

In [22]:
Y_test_age = vectorize_age(test_data)

model.evaluate(X_test, Y_test_age) 
model.save("drive/My Drive/gerald_data/W_age")

131703/131703 [==============================] - 26s 196us/sample - loss: 44.6770 - mean_absolute_error: 5.0201


Next we define the functions we will use to vectorize our data, which we can then feed into some models!

In [0]:
def vectorize_x(data):
  X = sp.sparse.lil_matrix((len(data), len(words)), dtype=np.int8)
  for i, post in enumerate(data):
    for word in list(set(post)):
      try: 
        X.rows[i].append(words.index(word)) # In a lil_matrix, there is self.rows, where the ith element is a list of column indexes where there are non-zero elements...
        X.data[i].append(1)                 # ...and self.data, where the ith element is a list of the non-zero elements that occur in that row.
      except: pass
    if i % 1000 == 0:
      print i
  return X.tocsr()
  

def vectorize_gender(data):
  return np.array(map(lambda post : 1 if post["gender"] == "male" else 0, data)) # Change genders to 1:male, 0:female

def vectorize_age(data):
  return np.array((post['age'] for post in data))

Here we apply SVD to reduce the dimensionality of the data to something more managable.

In [0]:
N_COMPONENTS = 300

svd = sk.decomposition.TruncatedSVD(N_COMPONENTS)
Xtr_vec = vectorize_x(posts)
X_train = svd.fit_transform(Xtr_vec) 
Xte_vec = vectorize_x([preprocess(post) for post in test_data])
X_test = svd.transform(Xte_vec)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000


# Models

In [0]:
Y_gender_train = vectorize_gender(train_data)
Y_gender_test = vectorize_gender(test_data)

bayes = BernoulliNB()
bayes.fit(Xtr_vec, Y_gender_train)
print "Naive bayes train accuracy: {:.2%}".format(bayes.score(Xtr_vec, Y_gender_train))
print "Naive bayes test accuracy: {:.2%}".format(bayes.score(Xte_vec, Y_gender_test))

#N_NEIGHBORS = 10
#nn = sk.neighbors.KNeighborsClassifier(N_NEIGHBORS)
#nn.fit(X_train, Y_gender_train)
#print "Nearest-neighbors train accuracy: {:.2%}".format(nn.score(X_train, Y_gender_train))
#print "Nearest-neighbors test accuracy: {:.2%}".format(nn.score(X_test, Y_gender_test))

TREES = 100
forest = sk.ensemble.RandomForestClassifier(n_estimators=TREES, max_depth=5)
forest.fit(Xtr_vec, Y_gender_train)
print "Random forest train accuracy: {:.2%}".format(forest.score(Xtr_vec, Y_gender_train))
print "Random forest test accuracy: {:.2%}".format(forest.score(Xte_vec, Y_gender_test))

Naive bayes train accuracy: 60.42%
Naive bayes test accuracy: 60.11%
Random forest train accuracy: 59.97%
Random forest test accuracy: 58.96%
